In [7]:
import pandas as pd


In [8]:
da1=pd.read_csv("dataset/crypto_data/LTC-USD.csv",names=["time","low","high","open","close","volume"])

In [9]:
da1.head(5)

,time,low,high,open,close,volume
0,1528968660,96.580002,96.589996,96.589996,96.580002,9.647200
1,1528968720,96.449997,96.669998,96.589996,96.660004,314.387024
2,1528968780,96.470001,96.570000,96.570000,96.570000,77.129799
3,1528968840,96.449997,96.570000,96.570000,96.500000,7.216067
4,1528968900,96.279999,96.540001,96.500000,96.389999,524.539978


In [10]:
main_df=pd.DataFrame()

ratios=["BCH-USD","BTC-USD","ETH-USD","LTC-USD"]
for ratio in ratios:
    print(ratio)

BCH-USD
BTC-USD
ETH-USD
LTC-USD


In [11]:
for ratio in ratios:
    print(ratio)

BCH-USD
BTC-USD
ETH-USD
LTC-USD


In [12]:
for ratio in ratios:
    dataset=f'dataset/crypto_data/{ratio}.csv'
    df=pd.read_csv(dataset,names=["time","low","high","open","close","volume"])
    df.rename(columns={"close": f"{ratio}_close", "volume": f"{ratio}_volume"}, inplace=True)

    df.set_index("time", inplace=True)  # set time as index so we can join them on this shared time
    df = df[[f"{ratio}_close", f"{ratio}_volume"]]  # ignore the other columns besides price and volume
    print(df.head())
    if len(main_df)==0:  # if the dataframe is empty
        main_df = df  # then it's just the current df
    else:  # otherwise, join this data to the main one
        main_df = main_df.join(df)

main_df.fillna(method="ffill", inplace=True)  # if there are gaps in data, use previously known values
main_df.dropna(inplace=True)
print(main_df.head())  # how did we do??

            BCH-USD_close  BCH-USD_volume
time                                     
1528968660     871.719971        5.675361
1528968720     870.859985       26.856577
1528968780     870.099976        1.124300
1528968840     870.789978        1.749862
1528968900     870.000000        1.680500
            BTC-USD_close  BTC-USD_volume
time                                     
1528968660    6489.549805        0.587100
1528968720    6487.379883        7.706374
1528968780    6479.410156        3.088252
1528968840    6479.410156        1.404100
1528968900    6479.979980        0.753000
            ETH-USD_close  ETH-USD_volume
time                                     
1528968720      486.01001       26.019083
1528968780      486.00000        8.449400
1528968840      485.75000       26.994646
1528968900      486.00000       77.355759
1528968960      486.00000        7.503300
            LTC-USD_close  LTC-USD_volume
time                                     
1528968660      96.580002        9

In [13]:
SEQ_LEN = 60  # how long of a preceeding sequence to collect for RNN
FUTURE_PERIOD_PREDICT = 3  # how far into the future are we trying to predict?
RATIO_TO_PREDICT = "LTC-USD"

In [33]:
def classify(current,future):
    if float(future) > float(current):
        return 1
    else:
        return 0

In [34]:
main_df['future']=main_df['LTC-USD_close'].shift(-FUTURE_PERIOD_PREDICT)

In [35]:
main_df['target']=list(map(classify,main_df['LTC-USD_close'],main_df['future']))

In [32]:
main_df.head(10)

,BCH-USD_close,BCH-USD_volume,BTC-USD_close,BTC-USD_volume,ETH-USD_close,ETH-USD_volume,LTC-USD_close,LTC-USD_volume,future,target
time,,,,,,,,,,
1528968720,870.859985,26.856577,6487.379883,7.706374,486.01001,26.019083,96.660004,314.387024,96.389999,0
1528968780,870.099976,1.124300,6479.410156,3.088252,486.00000,8.449400,96.570000,77.129799,96.519997,0
1528968840,870.789978,1.749862,6479.410156,1.404100,485.75000,26.994646,96.500000,7.216067,96.440002,0
1528968900,870.000000,1.680500,6479.979980,0.753000,486.00000,77.355759,96.389999,524.539978,96.470001,1
1528968960,869.989990,1.669014,6480.000000,1.490900,486.00000,7.503300,96.519997,16.991997,96.400002,0
1528969020,869.450012,0.865200,6477.220215,2.731950,485.98999,85.877251,96.440002,95.524078,96.400002,0
1528969080,869.989990,23.534929,6480.000000,2.174240,485.98999,160.915192,96.470001,175.205307,96.400002,0
1528969140,870.000000,2.300000,6479.990234,0.903100,485.98999,61.371887,96.400002,43.652802,96.400002,1
1528969200,870.320007,9.255514,6478.660156,3.258786,485.98999,42.687656,96.400002,8.160000,96.400002,1
